<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/C%C3%B3digos%20convencionales/Convolucional_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Convolucional 1/3

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import commpy.channelcoding.convcode as cc
import commpy.utilities as util
import scipy.stats as stats  # Importar scipy.stats

# Define la función BSC (Binary Symmetric Channel)
def bsc(input_bits, p_t):
    """
    Binary Symmetric Channel.

    Parameters
    ----------
    input_bits : 1D ndarray containing {0, 1}
        Input array of bits to the channel.

    p_t : float in [0, 1]
        Transition/Error probability of the channel.

    Returns
    -------
    output_bits : 1D ndarray containing {0, 1}
        Output bits from the channel.
    """
    output_bits = input_bits.copy()
    flip_locs = (np.random.rand(len(output_bits)) <= p_t)
    output_bits[flip_locs] = 1 ^ output_bits[flip_locs]
    return output_bits

# Define los parámetros del código convolucional
memory = np.array(3, ndmin=1)
g_matrix = np.array((0o7, 0o7, 0o5), ndmin=2)  # Tasa de codificación 1/2
trellis = cc.Trellis(memory, g_matrix)
tb_depth = 5 * (memory[0] + 1)  # Profundidad de traceback recomendada

# Probabilidades de error a probar
#error_probabilities = np.linspace(0.0001, 0.15, 20)

error_probabilities = np.array([0.134221053, 0.142110526, 0.15])



# Número de tramas y longitud de cada mensaje
num_block = 100000
batch_size = 100
num_messages_per_frame = 100
message_length = 50
num_epochs = 1

# Almacena los BER promedio para cada probabilidad de error
ber_results = {p_t: [] for p_t in error_probabilities}  # Diccionario para almacenar BER para cada probabilidad en cada época


for epoch in range(num_epochs):
    print("")
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(" ")

    for p_t in error_probabilities:
        total_bit_errors = 0
        total_bits = 0

        print(f"Simulating for error probability p_t={p_t}...")

        for _ in tqdm(range(int(num_block/batch_size))):
            # Generar bits de mensaje aleatorios en un batch
            message_bits_batch = np.random.randint(0, 2, (num_messages_per_frame, message_length))

            # Codificar los bits del mensaje en un batch
            coded_bits_batch = np.array([cc.conv_encode(message_bits, trellis) for message_bits in message_bits_batch])

            #print(coded_bits_batch.shape)

            # Pasar los bits codificados a través del canal BSC en un batch
            received_bits_batch = np.array([bsc(coded_bits, p_t) for coded_bits in coded_bits_batch])
            #print(received_bits_batch.shape)
            # Decodificar los bits recibidos en un batch
            decoded_bits_batch = np.array([cc.viterbi_decode(received_bits.astype(float), trellis, tb_depth) for received_bits in received_bits_batch])

            # Calcular el número de errores de bit en un batch
            for message_bits, decoded_bits in zip(message_bits_batch, decoded_bits_batch):
                num_bit_errors = util.hamming_dist(message_bits, decoded_bits[:len(message_bits)])
                total_bit_errors += num_bit_errors
                total_bits += len(message_bits)

        # Calcular la BER promedio para esta época
        ber = total_bit_errors / total_bits
        ber_results[p_t].append(ber)
        print(f"BER for p_t={p_t}: {ber}")

# Convertir los resultados en un DataFrame
results_df = pd.DataFrame({
    'Epoch': np.repeat(range(1, num_epochs + 1), len(error_probabilities)),
    'Probabilidad de Error (p_t)': np.tile(error_probabilities, num_epochs),
    'BER': [ber for sublist in ber_results.values() for ber in sublist]
})

# Guardar los resultados en un archivo CSV
results_df.to_csv('ber_vs_probabilidades22.csv', index=False)

# Calcular el promedio y el intervalo de confianza
mean_ber = []
conf_intervals = []

for p_t in error_probabilities:
    data = ber_results[p_t]
    mean = np.mean(data)
    confidence_interval = stats.t.interval(0.95, len(data)-1, loc=mean, scale=stats.sem(data))

    mean_ber.append(mean)
    conf_intervals.append(confidence_interval)

# Convertir listas a arrays para facilitar la manipulación
mean_ber = np.array(mean_ber)
conf_intervals = np.array(conf_intervals)

# Graficar los resultados
plt.figure()
plt.errorbar(error_probabilities, mean_ber,
             yerr=[mean_ber - conf_intervals[:, 0], conf_intervals[:, 1] - mean_ber],
             fmt='o', capsize=5)
plt.xlabel('Probabilidad de error (p_t)')
plt.ylabel('Tasa de Error de Bit (BER)')
plt.title('BER promedio vs Probabilidad de Error con intervalos de confianza')
plt.grid(True)
# plt.xscale('log')  # Escala logarítmica para el eje x si es necesario
plt.yscale('log')  # Escala logarítmica para el eje y
plt.show()

